In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [ ]:
fake_news = pd.read_csv('Fake.csv')
true_news = pd.read_csv('True.csv')

In [ ]:
true_news.head(2)

In [ ]:
fake_news.head(2)

In [ ]:
fake_news['class'] = 0
true_news['class'] = 1

In [ ]:
fake_news.shape,true_news.shape

In [ ]:
# Create a test dataframe with the last 10 rows of each dataset
# and drop the 10 columns from the original dataset

fake_news_manual_testing = fake_news.tail(10)

for i in range (23480, 23470, -1):
    fake_news.drop([i], axis=0, inplace = True)
    
true_news_manual_testing = true_news.tail(10)

for i in range (21416, 21406, -1):
    true_news.drop([i], axis=0, inplace = True)

In [ ]:
true_news_manual_testing.head(1)

In [ ]:
news_merge = pd.concat([fake_news, true_news], axis=0)
news_merge.head(2)

In [ ]:
# drop unnecessary columns
news = news_merge.drop(['title','subject','date'], axis=1)
news.head()

In [ ]:
# Give me the number of null values
news.isnull().sum()

In [ ]:
# Random shuffling of the data
news = news.sample(frac=1)
news.head()

In [ ]:
news.reset_index(inplace=True)
news.drop(['index'], axis=1, inplace=True)
news.head()

In [ ]:
# Function to process the text

def textprocessor(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('\\W', " ", text)
    text = re.sub("https?://\S+|www\.S+", "",text)
    text = re.sub(".*?>+", "", text)
    text = re.sub("[%s]" % re.escape(string.punctuation), "", text)
    text = re.sub("\n", "", text)
    text = re.sub("\w*\d\w*", "", text)
    return text

In [ ]:
news['text'] = news['text'].apply(textprocessor)

In [ ]:
x = news['text']
y = news['class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x,y,text_size=0.25)

In [ ]:
from sklean.feature_extraction.text import TfidVectorizer

In [ ]:
vectorization = TfidVectorizer()
xv_train = verization.fit_transform(X_train)
xv_test = verization.fit_transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

In [ ]:
pred = LR.predict(xv_test)

In [ ]:
score = LR.score(sv_test, y_test)

In [ ]:
print(classification_report(y_test,pred))